In [1]:
import os
import networkx as nx
import pandas as pd

# path = '/content/drive/My Drive/NEXCO/sample_code/matsunaga'
# os.chdir(path)
# print(os.getcwd())

In [7]:
# data directory
PROCESSED_DATA_DIR = '../Input_processed_data'
ORI_DATA_DIR = '../Input_original_data'

# IC, 道路情報 csv
IC_CSV = f'{PROCESSED_DATA_DIR}/road_master/ic_merged.csv'
IC_NET_SUB_CSV = f'{PROCESSED_DATA_DIR}/road_master/tateyama_kannetsu_doronet_sub.csv'

# 検索量 csv
SEARCH_COUNT_DIR = f'{PROCESSED_DATA_DIR}/search_count'
SEARCH_COUNT_KANNETSU_CSV = f'{SEARCH_COUNT_DIR}/search-count_kannetsu.csv'

In [8]:
df_ic = pd.read_csv(IC_CSV)
df_icnet_sub = pd.read_csv(IC_NET_SUB_CSV, dtype={'start_code': str, 'end_code': str})

code2name = dict(zip(df_ic['ic_code'], df_ic['ic_name']))

In [4]:
ic_subgraph = nx.from_pandas_edgelist(df_icnet_sub, source='start_code', target='end_code',
                                      edge_attr=['distance', 'road_code', 'direction'],
                                      create_using=nx.DiGraph())

## 検索データ, 交通データの補間 → マージ
### [注意点]
- このコードでは自分のローカル環境で既に前処理をした交通データを使用してしまっており、その構造に依存している箇所もあるため適宜変更してください

- 前処理済みの交通データの構造は以下を参照してください
  - datetime: 日時
  - start_name, end_name: IC名
  - start_code, end_code: ICコード
  - is_holiday: 休日フラグ
  - total: 実績データ内の「全車」に相当
  - speed: 実績データ内の「速度」に相当
  - KP: 実績データ内の「KP」に相当
  - OCC: 実績データ内の「OCC」に相当

- 前処理のコードは[sample_code/matsunaga/traffic_preprocess.ipynb](https://colab.research.google.com/drive/1ZfmGegTta-m4M2QofsRKAYxrjFa7txZG?usp=sharing)にありますので必要な際にご確認ください

In [9]:
ORI_DATA_DIR+'traffic/館山道（202104-202203）.CSV'

'../Input_original_datatraffic/館山道（202104-202203）.CSV'

交通実績データの読み込み

In [12]:
#df_kannetsu = pd.read_csv(ORI_DATA_DIR+'traffic/館山道（202104-202203）.CSV', index_col='datetime', parse_dates=True, dtype={'start_code': str, 'end_code': str})

df_kannetsu = pd.read_csv(ORI_DATA_DIR+'/traffic/館山道（202104-202203）.CSV',encoding='shift-jis', header=1)
df_kannetsu.reset_index(inplace=True)
df_kannetsu.head()

/opt/conda/envs/pycaret_full/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (22,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,index,支社,区分,道路,年,月,日,曜日,平休1,平休2（休日に土曜日含む）,...,車線率(追),OCC(追),速度(追),エラー数.2,全車(路),大車(路),車線率(路),OCC(路),速度(路),エラー数.3
0,0,関東支社,速度ループ,館山自動車道,2021,4,1,木,平,平,...,12.5,0.0,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,関東支社,速度ループ,館山自動車道,2021,4,1,木,平,平,...,0.0,0.0,111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,関東支社,速度ループ,館山自動車道,2021,4,1,木,平,平,...,22.2,0.0,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,関東支社,速度ループ,館山自動車道,2021,4,1,木,平,平,...,0.0,0.0,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,関東支社,速度ループ,館山自動車道,2021,4,1,木,平,平,...,0.0,0.0,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


検索データの読み込み

In [8]:
df_count_kannetsu = pd.read_csv(SEARCH_COUNT_KANNETSU_CSV, index_col='passing_time', parse_dates=True, dtype={'start_code': str, 'end_code': str})
df_count_kannetsu.reset_index(inplace=True)
df_count_kannetsu.head()

,passing_time,start_code,end_code,search
0,2021-04-02 00:00:00,1080291,1800186,0
1,2021-04-02 00:05:00,1080291,1800186,0
2,2021-04-02 00:10:00,1080291,1800186,0
3,2021-04-02 00:15:00,1080291,1800186,0
4,2021-04-02 00:20:00,1080291,1800186,0


### マージ時の補間処理
- ここでの補完方式は値をそのまま使っている
- なるべく早く動くようにしているためコードが読みにくいかもしれません

In [9]:
# 交通量データと検索データで区間情報が異なる区間に対応する行を交通量データからピックアップ
is_diff_sections = df_kannetsu[['start_code', 'end_code']].apply(lambda sec: not ic_subgraph.has_edge(sec[0], sec[1]), axis=1)
diff_sections = df_kannetsu.loc[is_diff_sections, ['start_code', 'end_code']].drop_duplicates().values

# 上の各区間について、検索データ（マスタデータ）に対応するように区間を分解する
broken_sections_dict = {}
for sec in diff_sections:
  p = nx.shortest_path(ic_subgraph, *sec)
  broken_sections_dict[tuple(sec)] = [(p[i], p[i+1]) for i in range(len(p)-1)]


# 交通量データ内の異なる区間情報を持つ各行について、その区間を分解されたものへと変更し、新たな行を作成する
new_rows = []
for _, row in df_kannetsu[is_diff_sections].iterrows():
  # 交通データ側の異なる区間を検索データ側の区間列に直す
  broken_sections = broken_sections_dict[(row['start_code'], row['end_code'])]

  for sec in broken_sections:
    new_r = row.to_list()
    new_r[3:5] = sec
    new_r[1:3] = code2name[sec[0]], code2name[sec[1]]
    new_rows.append(new_r)

df_new = pd.DataFrame(new_rows, columns=df_kannetsu.columns)

df_kannetsu_processed = pd.concat([df_kannetsu[~is_diff_sections], df_new]).sort_values('datetime').reset_index(drop=True)
df_kannetsu_processed

,datetime,start_name,end_name,start_code,end_code,is_holiday,total,speed,KP,OCC
0,2021-04-01 00:00:00,三芳ＰＡ,所沢,1800011,1800006,平,78.0,85.0,10.21,3.0
1,2021-04-01 00:00:00,前橋,駒寄ＰＡ,1800081,1800086,平,17.0,89.0,96.60,1.0
2,2021-04-01 00:00:00,高崎,前橋,1800076,1800081,平,32.0,88.0,88.20,1.0
3,2021-04-01 00:00:00,新座本線,所沢,1800004,1800006,平,57.0,87.0,3.90,2.0
4,2021-04-01 00:00:00,大泉ＪＣＴ,新座本線,1110210,1800004,平,57.0,87.0,3.90,2.0
...,...,...,...,...,...,...,...,...,...,...
10097563,2022-05-08 23:55:00,三芳ＰＡ,川越,1800011,1800016,休,55.0,96.0,16.38,2.0
10097564,2022-05-08 23:55:00,赤城,赤城高原ＳＡ,1800096,1800101,休,16.0,92.0,113.58,1.0
10097565,2022-05-08 23:55:00,赤城高原ＳＡ,昭和,1800101,1800106,休,16.0,92.0,113.58,1.0
10097566,2022-05-08 23:55:00,嵐山小川,東松山,1800041,1800036,休,51.0,94.0,44.50,2.0


### マージ
- 日付, ICコードをkeyとして交通実績側に左結合

In [10]:
df_merged_kannetsu = pd.merge(df_kannetsu_processed, df_count_kannetsu, 
                              how='left', left_on=['datetime', 'start_code', 'end_code'], 
                              right_on=['passing_time', 'start_code', 'end_code'])
df_merged_kannetsu.drop('passing_time', axis=1, inplace=True)
df_merged_kannetsu['search'].fillna(0, inplace=True)

df_merged_kannetsu

,datetime,start_name,end_name,start_code,end_code,is_holiday,total,speed,KP,OCC,search
0,2021-04-01 00:00:00,三芳ＰＡ,所沢,1800011,1800006,平,78.0,85.0,10.21,3.0,0.0
1,2021-04-01 00:00:00,前橋,駒寄ＰＡ,1800081,1800086,平,17.0,89.0,96.60,1.0,0.0
2,2021-04-01 00:00:00,高崎,前橋,1800076,1800081,平,32.0,88.0,88.20,1.0,0.0
3,2021-04-01 00:00:00,新座本線,所沢,1800004,1800006,平,57.0,87.0,3.90,2.0,0.0
4,2021-04-01 00:00:00,大泉ＪＣＴ,新座本線,1110210,1800004,平,57.0,87.0,3.90,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
10097563,2022-05-08 23:55:00,三芳ＰＡ,川越,1800011,1800016,休,55.0,96.0,16.38,2.0,0.0
10097564,2022-05-08 23:55:00,赤城,赤城高原ＳＡ,1800096,1800101,休,16.0,92.0,113.58,1.0,0.0
10097565,2022-05-08 23:55:00,赤城高原ＳＡ,昭和,1800101,1800106,休,16.0,92.0,113.58,1.0,0.0
10097566,2022-05-08 23:55:00,嵐山小川,東松山,1800041,1800036,休,51.0,94.0,44.50,2.0,1.0
